In [25]:
from bs4 import BeautifulSoup
import json
import requests
import pandas as pd
import numpy as np
import re
import requests

In [26]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        # Trova l'elemento span con la classe 'a-price'
        price_span = soup.find("span", class_='a-price')
        
        # Estrai il prezzo dall'elemento span
        price = price_span.find("span", class_='a-offscreen').text.strip()

    except AttributeError:
        price = ""

    return price

def get_rating(soup):
    try:
        # Trova la stringa che contiene il rating nel formato "x,x"
        rating_str = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    except AttributeError:
        try:
            rating_str = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating_str = ""

    # Rimuovi la virgola e converti la parte numerica in un valore float solo se rating_str contiene un valore numerico
    if rating_str:
        rating_str = rating_str.replace(',', '.')
        match = re.search(r'(\d+\.\d+)', rating_str)
        if match:
            rating = float(match.group(1))
            return rating
    
    # Restituisci None se non è stato trovato un rating valido
    return None

# Function to extract Number of User Reviews
def get_review_count1(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

def get_review_count(soup):
    try:
        review_count_str = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        review_count_str = ""

    # Rimuovi la virgola dalla stringa
    review_count_str = review_count_str.replace(',', '')
    review_count_str = review_count_str.replace('.', '')

    # Estrai la parte numerica dalla stringa
    review_count_numeric = re.search(r'(\d+)', review_count_str)

    # Se è stato trovato un valore numerico, restituiscilo come intero; altrimenti, restituisci None
    if review_count_numeric:
        return int(review_count_numeric.group(1))
    else:
        return None

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

def extract_dimensioni(soup):
    # Primo tentativo: utilizzando il codice HTML fornito inizialmente
    try:
        table = soup.find("table", {"id": "productDetails_techSpec_section_1"})
        rows = table.find_all("tr")
        for row in rows:
            header = row.find("th", class_="a-color-secondary a-size-base prodDetSectionEntry").text.strip()
            if "Dimensioni prodotto" in header:
                dimensioni_prodotto = row.find("td", class_="a-size-base prodDetAttrValue").text.strip()
                return dimensioni_prodotto
    except AttributeError:
        pass

    # Secondo tentativo: utilizzando il codice HTML fornito
    try:
        ul_element = soup.find("ul", class_="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list")
        li_elements = ul_element.find_all("li")
        for li in li_elements:
            bold_text = li.find("span", class_="a-text-bold").text.strip()
            if "Dimensioni prodotto" in bold_text:
                spans = li.find_all("span")
                if len(spans) > 2:
                    dimensioni_prodotto = spans[2].text.strip().split(';')[0].strip()
                    return dimensioni_prodotto
    except AttributeError:
        pass

    # Terzo tentativo: estrazione delle dimensioni basate su "cm" dal div fornito
    try:
        div_element = soup.find("div", {"id": "feature-bullets"})
        ul_element = div_element.find("ul", class_="a-unordered-list a-vertical a-spacing-mini")
        li_elements = ul_element.find_all("li")
        for li in li_elements:
            text_content = li.text
            match = re.search(r'(\d+ × \d+ × \d+ cm)', text_content)
            if match:
                return match.group(1)
    except (AttributeError, TypeError):
        pass

    # Quarto tentativo: estrazione delle dimensioni basate su "cm"
    try:
        description_content = soup.find("meta", attrs={'name':'description'})['content']
        index = description_content.find("cm")
        if index != -1:
            return description_content[index-13:index+2].strip()
    except (AttributeError, TypeError):
        pass

    # Quinto tentativo: estrazione delle dimensioni dello schermo dal tag meta description
    try:
        description_content = soup.find("meta", attrs={'name':'description'})['content']
        match = re.search(r'(\d+,\d+ Pollici)', description_content)
        if match:
            return match.group(1)
    except (AttributeError, TypeError):
        pass

    try:
        ul_element = soup.find("ul", class_="a-unordered-list a-nostyle a-vertical a-spacing-mini")
        li_elements = ul_element.find_all("li")
        for li in li_elements:
            text_content = li.text
            match = re.search(r'(\d+ x \d+ x \d+ cm)', text_content) 
            if match:
                return match.group(1)
    except (AttributeError, TypeError):
        pass

    try:
        ul_element = soup.find("ul", class_="a-unordered-list a-vertical a-spacing-mini")
        li_elements = ul_element.find_all("li")
        for li in li_elements:
            text_content = li.text
            match = re.search(r'(\d+ cm\*\d+ cm\*\d+ cm)', text_content)
            if match:
                return match.group(1)
    except (AttributeError, TypeError):
        pass

    # Se non troviamo le dimensioni in nessuno dei formati sopra, ritorniamo una stringa vuota
    return ""

def scrape_colore(soup):
    colori_trovati = []

    try:
        # Trova la tabella con le specifiche del prodotto
        table = soup.find("table", {"id": "productDetails_techSpec_section_1"})
        
        # Estrai tutte le righe della tabella
        rows = table.find_all("tr")

        # Itera su ogni riga per trovare quella con il "colore"
        for row in rows:
            header = row.find("th", class_="a-color-secondary a-size-base prodDetSectionEntry").text.strip()
            if header == "colore":
                colore_tabella = row.find("td", class_="a-size-base prodDetAttrValue").text.strip()
                colori_trovati.append(colore_tabella)

    except AttributeError:
        pass

    # Se non trova il colore nella tabella, cerca nel testo della pagina
    colori = ["grigio", "nero", "bianco", "rosa", "giallo", "rosso", "verde", "blu", "marrone", "arancione", "viola", "azzurro"]
    text = soup.get_text().lower()  # Ottieni tutto il testo della pagina e convertilo in minuscolo
    for colore in colori:
        if colore in text and colore not in colori_trovati:
            colori_trovati.append(colore)

    return colori_trovati

def scrape_peso_articolo(soup):
    weights = []

    try:
        # Trova la tabella con le specifiche del prodotto
        table = soup.find("table", {"id": "productDetails_techSpec_section_1"})
        
        # Estrai tutte le righe della tabella
        rows = table.find_all("tr")

        # Itera su ogni riga per trovare quella con il "Peso articolo"
        for row in rows:
            header = row.find("th", class_="a-color-secondary a-size-base prodDetSectionEntry").text.strip()
            if header == "Peso articolo":
                weights.append(row.find("td", class_="a-size-base prodDetAttrValue").text.strip())

    except AttributeError:
        pass

    # Nuovo tentativo: estrazione del peso basata su "grammi", "kg", "g"
    try:
        ul_element = soup.find("ul", class_="a-unordered-list a-vertical a-spacing-mini")
        li_elements = ul_element.find_all("li")
        for li in li_elements:
            text_content = li.text
            match = re.search(r'((\d+[.,])?\d+\s?(grammi|g|kg|kilograms))', text_content)
            if match:
                weights.append(match.group(1))
            
    except (AttributeError, TypeError):
        pass

    # Se non troviamo il peso in nessuno dei formati sopra, ritorniamo una lista vuota
    return weights

def extract_weights(weight_list):
    numeric_values = []
    for weight_str in weight_list:
        match = re.search(r'(\d+[.,]\d+|\d+)', weight_str)
        if match:
            numeric_value_str = match.group(1).replace(',', '.')
            numeric_value = float(numeric_value_str)
            if 'g' in weight_str or 'grammi' in weight_str:
                numeric_values.append(numeric_value)  # Aggiungi il valore numerico alla lista se ci sono g/grammi
            elif 'kg' in weight_str or 'kilograms' in weight_str:
                numeric_values.append(numeric_value * 1000)  # Aggiungi il valore convertito in grammi alla lista se ci sono kg
    return numeric_values



In [27]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36', 'Accept-Language': 'it-IT, it;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.it/s?k=sgrassatori+spray&page=4&__mk_it_IT=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=JAY03S5DCEYW&qid=1700828974&sprefix=sgrassatore+spray%2Caps%2C123&ref=sr_pg_4"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    #links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
    print(links)

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    #for link in links:
        #links_list.append(link.get('href'))
    
    # Loop for extracting links from Tag Objects
    for link in links:
        href = link.get('href')
        if not href.startswith("https://"):
            links_list.append(href)


    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[], "colore":[], "Dimensione prodotto":[], "peso_articolo":[], "link":[]}
    

    print(type(links_list))

    # Loop for extracting product details from each link 
    for link in links_list:
        print(link)
        new_webpage = requests.get("https://www.amazon.it" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        d['colore'].append(scrape_colore(new_soup))
        d['Dimensione prodotto'].append(extract_dimensioni(new_soup))
        #Peso
        peso_numerico_list = scrape_peso_articolo(new_soup)
        print(peso_numerico_list)
        pesi = extract_weights(peso_numerico_list)
        pesi_filtrati = [p for p in pesi if 50 <= p <= 3000]
        peso = pesi_filtrati[0] if pesi_filtrati else np.nan
        d['peso_articolo'].append(peso)
        #link
        d['link'].append(new_webpage.url)


    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_review.csv", header=True, index=False)


[<a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/sspa/click?ie=UTF8&amp;spc=MTo1MzU2NjYwMTY4ODcyNjkyOjE3MDA4MjkzNTg6c3BfYXRmX25leHQ6MjAwNzA3OTY3OTk2MDE6OjA6Og&amp;url=%2FNapisan-Igienizzante-Superfici-Limone-Confezione%2Fdp%2FB018U6VDRE%2Fref%3Dsr_1_145_sspa%3F__mk_it_IT%3D%25C3%2585M%25C3%2585%25C5%25BD%25C3%2595%25C3%2591%26crid%3DJAY03S5DCEYW%26keywords%3Dsgrassatori%2Bspray%26qid%3D1700829358%26sprefix%3Dsgrassatore%2Bspray%252Caps%252C123%26sr%3D8-145-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1"><span class="a-size-base-plus a-color-base a-text-normal">Napisan Spray Disinfettante Multisuperfici, Limone e Menta, Confezione da 12 Spray Disinfettanti, Spray da 750 ml</span> </a>, <a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/sspa/click?ie=UTF8&amp;spc=MTo1MzU2NjYwMTY4ODcyNjkyOjE3MDA4MjkzNTg6c3BfYXRmX25leHQ6MjAwMzgwMjEzMDc1OTg6OjA6Og&amp;url=%2FBactHome-materassi-attrezz

In [28]:
amazon_df

,title,price,rating,reviews,availability,colore,Dimensione prodotto,peso_articolo,link
0,"Napisan Spray Disinfettante Multisuperfici, Li...","23,99€",4.7,197.0,Disponibilità immediata,[bianco],"35,2 x 23,2 x 28,7 cm",NaN,https://www.amazon.it/Napisan-Igienizzante-Sup...
1,"BactPro - BactHome - elimina odori, macchie di...","15,99€",4.3,913.0,Disponibilità immediata,"[nero, rosa, giallo]","‎5,5 x 10 x 30 cm; 850 grammi",NaN,https://www.amazon.it/BactHome-materassi-attre...
2,"Fulcron Sgrassatore Pronto all'Uso, Super Sgra...","7,50€",4.4,165.0,Disponibilità immediata,[],"‎11,3 x 5,2 x 27,3 cm; 891,66 grammi",892.0,https://www.amazon.it/Sgrassatore-Professional...
3,ECASAN Spray Igienizzante Multiuso Spray Disin...,"12,90€",4.5,67.0,Disponibilità immediata,[],,NaN,https://www.amazon.it/Igienizzante-Ambienti-Mu...
4,Rhütten Azione Disodorante Pulitore Schiumogen...,"6,69€",4.0,856.0,Disponibilità immediata,[],‎6 x 6 x 25 cm; 440 grammi,440.0,https://www.amazon.it/Rhutten-180716-Pulitore-...
...,...,...,...,...,...,...,...,...,...
60,"Chanteclair Sgrassatore con candeggina, 625 ML","€1,99",4.6,3.0,Not Available,[],,NaN,https://www.amazon.it/Chanteclair-Sgrassatore-...
61,"HG Detergente per Forni, Grill e Barbecue, Rim...","6,99€",3.9,83.0,Disponibilità immediata,[],"‎6,7 x 6,7 x 24,9 cm; 570 grammi",NaN,https://www.amazon.it/HG-detergente-forni-gril...
62,Cillit Bang Sgrassatore potente Spray 750 ml –...,"25,31€",4.6,270.0,Disponibilità: solo 4,[],"‎9,6 x 6,4 x 28,8 cm; 2,15 Kg",NaN,https://www.amazon.it/Cillit-Bang-Sgrassatore-...
63,Citrosil Home Protection - Spray Disinfettante...,,4.6,4558.0,Not Available,"[giallo, blu]","‎6,5 x 6,5 x 20,1 cm; 280 grammi",NaN,https://www.amazon.it/Citrosil-Home-Protection...


FATTO LO SCRAPING DEI PRODOTTI

In [29]:
amazon_df['punteggio'] = amazon_df['rating'] * np.log10(amazon_df['reviews'] + 1)
punteggio_col = amazon_df.pop('punteggio')
amazon_df.insert(4, 'punteggio', punteggio_col)

#dividi in 6 quantili in base al punteggio
amazon_df['punteggio_quintile'] = pd.qcut(amazon_df['punteggio'], 5, labels=['0-20%', '20-40%', '40-60%', '60-80%', '80-100%'])
punteggio_col = amazon_df.pop('punteggio_quintile')
amazon_df.insert(5, 'punteggio_quintile', punteggio_col)
amazon_df = amazon_df.drop(columns=['rating', 'reviews'])

amazon_df['Dimensione prodotto'] = amazon_df['Dimensione prodotto'].str.replace(r'(cm;).*$', 'cm', regex=True)
amazon_df['Dimensione prodotto'] = amazon_df['Dimensione prodotto'].str.replace(r'(cm).*$', 'cm', regex=True)

In [30]:
amazon_df

,title,price,punteggio,punteggio_quintile,availability,colore,Dimensione prodotto,peso_articolo,link
0,"Napisan Spray Disinfettante Multisuperfici, Li...","23,99€",10.794326,60-80%,Disponibilità immediata,[bianco],"35,2 x 23,2 x 28,7 cm",NaN,https://www.amazon.it/Napisan-Igienizzante-Sup...
1,"BactPro - BactHome - elimina odori, macchie di...","15,99€",12.732069,60-80%,Disponibilità immediata,"[nero, rosa, giallo]","‎5,5 x 10 x 30 cm",NaN,https://www.amazon.it/BactHome-materassi-attre...
2,"Fulcron Sgrassatore Pronto all'Uso, Super Sgra...","7,50€",9.768476,40-60%,Disponibilità immediata,[],"‎11,3 x 5,2 x 27,3 cm",892.0,https://www.amazon.it/Sgrassatore-Professional...
3,ECASAN Spray Igienizzante Multiuso Spray Disin...,"12,90€",8.246290,40-60%,Disponibilità immediata,[],,NaN,https://www.amazon.it/Igienizzante-Ambienti-Mu...
4,Rhütten Azione Disodorante Pulitore Schiumogen...,"6,69€",11.731923,60-80%,Disponibilità immediata,[],‎6 x 6 x 25 cm,440.0,https://www.amazon.it/Rhutten-180716-Pulitore-...
...,...,...,...,...,...,...,...,...,...
60,"Chanteclair Sgrassatore con candeggina, 625 ML","€1,99",2.769476,0-20%,Not Available,[],,NaN,https://www.amazon.it/Chanteclair-Sgrassatore-...
61,"HG Detergente per Forni, Grill e Barbecue, Rim...","6,99€",7.504689,20-40%,Disponibilità immediata,[],"‎6,7 x 6,7 x 24,9 cm",NaN,https://www.amazon.it/HG-detergente-forni-gril...
62,Cillit Bang Sgrassatore potente Spray 750 ml –...,"25,31€",11.191659,60-80%,Disponibilità: solo 4,[],"‎9,6 x 6,4 x 28,8 cm",NaN,https://www.amazon.it/Cillit-Bang-Sgrassatore-...
63,Citrosil Home Protection - Spray Disinfettante...,,16.830800,80-100%,Not Available,"[giallo, blu]","‎6,5 x 6,5 x 20,1 cm",NaN,https://www.amazon.it/Citrosil-Home-Protection...


In [31]:
#quali e quanti sono i valori in Dimensione prodotto?
amazon_df['peso_articolo'].value_counts()

peso_articolo
500.0    2
892.0    1
440.0    1
400.0    1
330.0    1
300.0    1
570.0    1
580.0    1
590.0    1
970.0    1
Name: count, dtype: int64

In [32]:
# Assicurati che la colonna 'link' esista
if 'link' in amazon_df.columns:
    # Aggiungi i link al file di testo esistente
    with open('link.txt', 'a') as file:
        for link in amazon_df['link']:
            file.write(link + "\n")
else:
    print("La colonna 'link' non esiste nel dataset.")

In [ ]:
def filter_links(file_path, brands):
    # Leggere il contenuto del file
    with open(file_path, 'r') as file:
        links = file.readlines()

    # Filtrare i link che contengono le marche specificate
    filtered_links = [link for link in links if any(brand.lower() in link.lower() for brand in brands)]

    # Sovrascrivere il file con i link filtrati
    with open(file_path, 'w') as file:
        for link in filtered_links:
            file.write(link)

# Percorso del file
file_path = 'link.txt'  # Modifica con il percorso corretto del tuo file

# Lista delle marche da cercare (case insensitive)
brands = ['Chanteclair', 'Ace', 'Smac', 'Winnis', 'Amuchina', 'Cif', 'Quasar', 'Vetril']

# Chiamare la funzione per filtrare i link
filter_links(file_path, brands)